In [1]:
from web3 import Web3
import json
import pandas as pd
import time
import datetime

ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

if not web3.is_connected():
    print("❌ Could not connect. Make sure Ganache is running.")
    exit(1)
print("✅ Connected to Ganache.")

contract_address = "0x1d7aF1c683F8093A4baBE4B7Ee5D5E1ebeb8C063"

abi_json = """
[
  {
    "inputs": [],
    "stateMutability": "nonpayable",
    "type": "constructor"
  },
  {
    "anonymous": false,
    "inputs": [
      {
        "indexed": false,
        "internalType": "uint256",
        "name": "timestamp",
        "type": "uint256"
      },
      {
        "indexed": false,
        "internalType": "string",
        "name": "rfid",
        "type": "string"
      },
      {
        "indexed": false,
        "internalType": "string",
        "name": "gpsSensor",
        "type": "string"
      },
      {
        "indexed": false,
        "internalType": "string",
        "name": "temperatureSensor",
        "type": "string"
      }
    ],
    "name": "DataStored",
    "type": "event"
  },
  {
    "inputs": [
      {
        "internalType": "string",
        "name": "_rfid",
        "type": "string"
      },
      {
        "internalType": "string",
        "name": "_gpsSensor",
        "type": "string"
      },
      {
        "internalType": "string",
        "name": "_temperatureSensor",
        "type": "string"
      }
    ],
    "name": "storeData",
    "outputs": [],
    "stateMutability": "nonpayable",
    "type": "function"
  },
  {
    "inputs": [
      {
        "internalType": "uint256",
        "name": "",
        "type": "uint256"
      }
    ],
    "name": "dataRecords",
    "outputs": [
      {
        "internalType": "uint256",
        "name": "timestamp",
        "type": "uint256"
      },
      {
        "internalType": "string",
        "name": "rfid",
        "type": "string"
      },
      {
        "internalType": "string",
        "name": "gpsSensor",
        "type": "string"
      },
      {
        "internalType": "string",
        "name": "temperatureSensor",
        "type": "string"
      }
    ],
    "stateMutability": "view",
    "type": "function"
  },
  {
    "inputs": [
      {
        "internalType": "uint256",
        "name": "index",
        "type": "uint256"
      }
    ],
    "name": "getRecord",
    "outputs": [
      {
        "internalType": "uint256",
        "name": "",
        "type": "uint256"
      },
      {
        "internalType": "string",
        "name": "",
        "type": "string"
      },
      {
        "internalType": "string",
        "name": "",
        "type": "string"
      },
      {
        "internalType": "string",
        "name": "",
        "type": "string"
      }
    ],
    "stateMutability": "view",
    "type": "function"
  },
  {
    "inputs": [],
    "name": "getTotalRecords",
    "outputs": [
      {
        "internalType": "uint256",
        "name": "",
        "type": "uint256"
      }
    ],
    "stateMutability": "view",
    "type": "function"
  },
  {
    "inputs": [],
    "name": "MAX_ENTRIES",
    "outputs": [
      {
        "internalType": "uint256",
        "name": "",
        "type": "uint256"
      }
    ],
    "stateMutability": "view",
    "type": "function"
  },
  {
    "inputs": [],
    "name": "owner",
    "outputs": [
      {
        "internalType": "address",
        "name": "",
        "type": "address"
      }
    ],
    "stateMutability": "view",
    "type": "function"
  }
]
"""

abi = json.loads(abi_json)

contract = web3.eth.contract(address=contract_address, abi=abi)

web3.eth.default_account = web3.eth.accounts[0]
print(f"✅ Connected to Smart Contract at {contract_address}")
print(f"Using default account: {web3.eth.default_account}\n")

df = pd.read_csv("logistics_tracking_data.csv")

print("CSV Preview:")
print(df.head(), "\n")

def send_iot_data(rfid: str, gps_sensor: str, temperature_sensor: str):
    """
    Sends a single IoT data record to the deployed smart contract.
    """
    
    txn = contract.functions.storeData(rfid, gps_sensor, temperature_sensor).transact({
        'from': web3.eth.default_account,
        'gas': 3_000_000
    })
    
    receipt = web3.eth.wait_for_transaction_receipt(txn)
    print(f"✅ Stored | RFID: {rfid} | GPS Sensor: {gps_sensor} | Temp Sensor: {temperature_sensor} | Txn: {receipt.transactionHash.hex()}")


print("⏳ Sending CSV rows to blockchain...\n")
for idx, row in df.iterrows():

    dev_id    = str(row["rfid"])
    dtype     = str(row["gps_sensor"])
    dvalue    = str(row["temperature_sensor"])

    send_iot_data(dev_id, dtype, dvalue)

    time.sleep(1)

print("\n✅ Finished sending all CSV rows.\n")

total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")

first = contract.functions.getRecord(0).call()
ts0, d0, t0, v0 = first
print("\nFirst stored record:")
print(f"  timestamp: {datetime.datetime.utcfromtimestamp(ts0).strftime('%Y-%m-%d %H:%M:%S UTC')}")
print(f"  rfid : {d0}")
print(f"  gpsSensor : {t0}")
print(f"  temperatureSensor: {v0}")

last_index = total_records - 1
last = contract.functions.getRecord(last_index).call()
tsL, dL, tL, vL = last
print("\nLast stored record:")
print(f"  timestamp: {datetime.datetime.utcfromtimestamp(tsL).strftime('%Y-%m-%d %H:%M:%S UTC')}")
print(f"  rfid : {dL}")
print(f"  gpsSensor : {tL}")
print(f"  temperatureSensor: {vL}")

✅ Connected to Ganache.
✅ Connected to Smart Contract at 0x1d7aF1c683F8093A4baBE4B7Ee5D5E1ebeb8C063
Using default account: 0x68bFF90EAa76fF85F670408b8B2fc83d3752E01f

CSV Preview:
                    timestamp    rfid  gps_sensor  temperature_sensor
0  2025-05-08 06:37:34.786672  TRK716          61                20.4
1  2025-05-07 23:56:34.795947  TRK578          76                24.6
2  2025-05-08 07:54:34.796037  TRK434          63                23.1
3  2025-05-08 02:26:34.796065  TRK230          79                23.7
4  2025-05-07 22:51:34.796086  TRK593          62                23.8 

⏳ Sending CSV rows to blockchain...

✅ Stored | RFID: TRK716 | GPS Sensor: 61 | Temp Sensor: 20.4 | Txn: 01afb292774841c70eaada1d0f2b47f994e1daf80fa8575af5f0f679851c4e07
✅ Stored | RFID: TRK578 | GPS Sensor: 76 | Temp Sensor: 24.6 | Txn: 56ba9783dbb24b725668d4d1d1e382a4b4b0c52b8e97021820c778a7f34c7a49
✅ Stored | RFID: TRK434 | GPS Sensor: 63 | Temp Sensor: 23.1 | Txn: 07432ae149eb59a04d33bfb9822

Web3RPCError: {'message': 'VM Exception while processing transaction: revert Storage limit reached', 'stack': 'RuntimeError: VM Exception while processing transaction: revert Storage limit reached\n    at EIP1559FeeMarketTransaction.fillFromResult (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache/dist/node/1.js:2:12745)\n    at Miner.<anonymous> (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache/dist/node/1.js:2:36703)\n    at async Miner.<anonymous> (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache/dist/node/1.js:2:35116)\n    at async Miner.mine (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache/dist/node/1.js:2:39680)\n    at async Blockchain.mine (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache/dist/node/1.js:2:60063)\n    at async Promise.all (index 0)\n    at async TransactionPool.emit (/Applications/Ganache.app/Contents/Resources/static/

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from datetime import datetime, timedelta


In [ ]:
num_records = 100  # Adjust this number as needed


# Example for Smart Logistics Tracking
data = []

In [ ]:
for _ in range(num_records):
    record = {
        "timestamp": datetime.now() - timedelta(minutes=np.random.randint(0, 1440)),  # Random timestamp in the last 24 hours
        "rfid": f"TRK{np.random.randint(100, 999)}",  # Random RFID
        "gps_sensor": np.random.randint(60, 100),  # Normal heart rate range
        "temperature_sensor": round(np.random.uniform(20.0, 25.0), 1)  # Goods temperature in Celsius
    }
    data.append(record)

In [ ]:
# Convert to DataFrame
df = pd.DataFrame(data)

In [ ]:
print(df.temperature_sensor)

0     20.4
1     24.6
2     23.1
3     23.7
4     23.8
      ... 
95    24.9
96    22.9
97    22.8
98    24.9
99    24.0
Name: temperature_sensor, Length: 100, dtype: float64


In [ ]:
df.to_csv("logistics_tracking_data.csv", index=False)
df.to_json("logistics_tracking_dat", orient="records")